In [ ]:
class Individual:
    """
    Our Individual is a neural network
    code - flattened weights and biases
    fitness - performance of the NN on a dataset
    """

    def __init__(self, input_size, hidden_size, output_size, X, y):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.X = X  # input data
        self.y = y  # target data

        # Initialize weights and biases randomly
        self.W1 = np.random.randn(input_size, hidden_size)
        self.b1 = np.random.randn(hidden_size)
        self.W2 = np.random.randn(hidden_size, output_size)
        self.b2 = np.random.randn(output_size)

        # Flatten all weights into a code for GA
        self.code = self.flatten_weights()
        self.calc_fitness()

    def flatten_weights(self):
        """Flatten all weights and biases into a 1D array"""
        return np.concatenate([
            self.W1.flatten(),
            self.b1.flatten(),
            self.W2.flatten(),
            self.b2.flatten()
        ])

    def unflatten_weights(self):
        idx = 0
        W1_size = self.input_size * self.hidden_size
        self.W1 = self.code[idx:idx+W1_size].reshape(self.input_size, self.hidden_size)
        idx += W1_size

        self.b1 = self.code[idx:idx+self.hidden_size]
        idx += self.hidden_size

        W2_size = self.hidden_size * self.output_size
        self.W2 = self.code[idx:idx+W2_size].reshape(self.hidden_size, self.output_size)
        idx += W2_size

        self.b2 = self.code[idx:idx+self.output_size]

    def forward(self, X):
        """Feedforward pass"""
        h = np.tanh(np.dot(X, self.W1) + self.b1)
        logits = np.dot(h, self.W2) + self.b2
        # Softmax for probabilities
        exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))
        probs = exp_logits / np.sum(exp_logits, axis=1, keepdims=True)
        return probs

    def calc_fitness(self):
        self.unflatten_weights()
        probs = self.forward(self.X)  # compute predicted probabilities
        y_pred = np.argmax(probs, axis=1)  # predicted class for each sample
        accuracy = np.mean(y_pred == self.y)  # fraction of correct predictions
        self.fitness = accuracy  # higher is better

In [ ]:
def selection(population: list, k: int, method: str = "tournament"):
    """
    Selects an individual from the population based on fitness.

    Parameters:
        population - list of Individuals (with NN code & fitness)
        k          - tournament size (for tournament selection)
        method     - either "tournament" or "roulette"
    """
    if method == "tournament":
        # Tournament selection
        k = min(len(population), k)
        participants = random.sample(population, k)
        return max(participants, key=lambda x: x.fitness)

    # TODO: add roulette


In [ ]:
def crossover(parent1, parent2, child1, child2):
    # Pick a random point in the flattened code array
    breakpoint = random.randrange(1, len(parent1.code))

    # Mix the weights
    child1.code[:breakpoint] = parent1.code[:breakpoint]
    child1.code[breakpoint:] = parent2.code[breakpoint:]

    child2.code[:breakpoint] = parent2.code[:breakpoint]
    child2.code[breakpoint:] = parent1.code[breakpoint:]

In [ ]:
def mutation(child: Individual, p: float, sigma: float = 0.1):
    """
    With probability p, add Gaussian noise to each weight in the network.

    p     - probability of mutating each weight
    sigma - standard deviation of noise
    """
    for i in range(len(child.code)):
        if random.random() < p:
            child.code[i] += np.random.normal(0, sigma)

In [ ]:
from copy import deepcopy
from matplotlib import pyplot as plt
import numpy as np
import random

def ga(X, y, input_size, hidden_size, output_size,
          population_size=20, num_generations=50,
          tournament_size=3, mutation_prob=0.1, elitism_size=2):

    """
    Genetic Algorithm for evolving a neural network

    X, y           : training data and labels
    input_size      : number of input features
    hidden_size     : number of neurons in hidden layer
    output_size     : number of output classes
    population_size : individuals per generation
    num_generations : GA iterations
    tournament_size : tournament selection size
    mutation_prob   : probability of mutating each weight
    elitism_size    : number of best individuals copied to next generation
    """

    # Initialize population
    population = [Individual(input_size, hidden_size, output_size, X, y) for _ in range(population_size)]
    new_population = [Individual(input_size, hidden_size, output_size, X, y) for _ in range(population_size)]

    # Adjust elitism to match parity with population size
    if elitism_size % 2 != population_size % 2:
        elitism_size += 1

    best_fitnesses = []

    for generation in range(num_generations):
        # Sort population by fitness (accuracy)
        population.sort(key=lambda x: x.fitness, reverse=True)
        best_fitnesses.append(population[0].fitness)
        print(f"Generation {generation}: Best accuracy = {population[0].fitness:.4f}")

        # Elitism: carry best individuals directly to new population
        new_population[:elitism_size] = deepcopy(population[:elitism_size])

        # Fill the rest of the new population via crossover + mutation
        for i in range(elitism_size, population_size, 2):
            parent1 = selection(population, tournament_size)
            # temporarily set fitness to -inf to avoid selecting same parent
            tmp, parent1.fitness = parent1.fitness, float('-inf')
            parent2 = selection(population, tournament_size)
            parent1.fitness = tmp

            # Create children via crossover
            crossover(parent1, parent2, new_population[i], new_population[i+1])

            # Mutate children
            mutation(new_population[i], mutation_prob)
            mutation(new_population[i+1], mutation_prob)

            # Evaluate fitness (accuracy)
            new_population[i].calc_fitness()
            new_population[i+1].calc_fitness()

        # Replace old population
        population = deepcopy(new_population)

    # Return best individual
    best_individual = max(population, key=lambda x: x.fitness)
    print(f'\nBest accuracy: {best_individual.fitness:.4f}')
    plt.plot(best_fitnesses)
    plt.xlabel('Generation')
    plt.ylabel('Best Accuracy')
    plt.title('GA Neural Network Evolution')
    plt.show()

    return best_individual


In [ ]:
import pandas as pd

df = pd.read_csv('../data/dataset.csv')
df.head()

In [ ]:
X = df.drop(columns=['label', 'user_id', 'destination', 'dest_key'])
X.head()


In [ ]:
y = df['label']
y.unique()

In [ ]:
df['user_id'].head()

In [ ]:
from sklearn.model_selection import GroupShuffleSplit

"""
As we can see above, there are multiple rows per user, so we have to make sure that all data from the same user stays
together in either train, validation, or test set. GroupShuffleSplit is going to ensure exactly that.
"""

groups = df['user_id']

# First, we split the data into train set and the rest
split1 = GroupShuffleSplit(n_splits=1, test_size=0.30, random_state=42)
train_idx, other_idx = next(split1.split(X, y, groups=groups))

X_train, X_other = X.iloc[train_idx], X.iloc[other_idx]
y_train, y_other = y.iloc[train_idx], y.iloc[other_idx]
other_groups = groups.iloc[other_idx]

# Then we split the rest of the data into validation and test sets
split2 = GroupShuffleSplit(n_splits=1, test_size=0.50, random_state=42)
val_idx, test_idx = next(split2.split(X_other, y_other, groups=other_groups))

X_val, X_test = X_other.iloc[val_idx], X_other.iloc[test_idx]
y_val, y_test = y_other.iloc[val_idx], y_other.iloc[test_idx]

print(X_train.shape, X_val.shape, X_test.shape)

In [ ]:
categorical_cols = ['season', 'activity_level', 'safety_conscious', 'popularity', 'traveller_type']
numeric_cols = [c for c in X.columns if c not in categorical_cols]
numeric_cols

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
        ('num', 'passthrough', numeric_cols)                                    # leave numeric columns unchanged
    ]
)

X_train_final = preprocessor.fit_transform(X_train)
X_val_final   = preprocessor.transform(X_val)
X_test_final  = preprocessor.transform(X_test)

X_train_final

In [ ]:
import numpy as np

input_size = X_train_final.shape[1]         # 71 features
hidden_size = 32                            # hidden layer neurons
output_size = len(np.unique(y))             # 2 classes
population_size = 50
num_generations = 50
tournament_size = 3
mutation_prob = 0.2
elitism_size = 2

best_nn = ga(
    X=X_train_final,
    y=y_train,
    input_size=input_size,
    hidden_size=hidden_size,
    output_size=output_size,
    population_size=population_size,
    num_generations=num_generations,
    tournament_size=tournament_size,
    mutation_prob=mutation_prob,
    elitism_size=elitism_size
)

y_pred_test = np.argmax(best_nn.forward(X_test_final), axis=1)
accuracy_test = np.mean(y_pred_test == y_test)
print(f"\nTest set accuracy: {accuracy_test:.4f}")
